<img src="../images/AzPTravel_PPM.png">

## Read in Data Collection Source

#### This script fetches the current data collection submission file from the completed "prep.ipynb" step into the pipeline.


### User Variables
- These are overwritten if inherited from run_control.ipynb.
- Feel Free to reset them for a manual run if you like
- Do not save without percode = "-f"

In [ ]:
commit_message = "Development and testing."
# Give a brief reason for the run.

run_control = 0
#run_type = 0 - Lite run with no reporting, not recommended.
#run_type = 1 - Lite run with normal reporting, default setting.
#run_type = 2 - Heavy run with full reporting, available for audits and troubleshooting.
#run_type = 5 - A default setting. Indicates the script is being run by an outside process without an inherited value

percode = "2021.Q1"
# Data Collection Code, this controls file paths and output names
# "-f" is the value indicating a bad inheritance from run with arg

#----------
# do not edit - this either inherits the full instance timestamp from the papermill book or captures the run time of this script.
from datetime import datetime
inst_datetime = datetime.now().strftime("%m%d%Y%H%M%S")

In [ ]:
# Parameters
run_control = 1
percode = "2021.Q1"
commit_message = "Edited data for Austraila, json 2 parquet and add comments from email. Begin general evaluation of data submissions."
inst_datetime = "05242021184739"


#### import packages

In [ ]:
import os # System commands
import sys # System commands

import getpass # parquet file read/write
import json # json file read/write
from sklearn import datasets # pass vars between .ipynb s

import matplotlib.pyplot as plt #Plots and Graphs
import pandas as pd #DataFrame and math


#### Default Variables, these govern logic, do not edit.

In [ ]:
default_dc = "20XX.QX"
default_rc = 0 #extra lite mode
dummy_perc = "33Q3" # bad inheritance

#### Script determining run context ie, manual, run_control.ipynb, or other.

In [ ]:
if run_control == 5:
    
    run_control = default_rc
    
else:
    
    run_control = run_control

try:
    
    if sys.argv[1] == "-f":
        
        percode = percode
        
    else:
        
        percode = sys.argv[1]

except IndexError:
    
    percode = default_dc

except NameError:

    percode = default_dc


#### style settings

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### Make paths for the source folder

In [ ]:
rt_path = f'//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/{str(percode)}'


#### Set file variables

In [ ]:
prepparq = os.path.join(str(rt_path), f"{percode}prep.parquet")
prepfile = os.path.join(str(rt_path), f"{percode}prep.csv")


- Read each json file into temp DataFrame "data".
- Prep actions - Strip (trim) leading and trailing spaces values in string columns, remove rows without business units.
- Append each json output to into 1 DataFrame "df".
- Process Cleanup, the indices restart with each append, reset and drop the previous.

In [ ]:

try:
    %store -r prep_df
    df = prep_df

except:
    
    try:
        df = pd.read_parquet(prepparq, engine = "pyarrow")
    
    except:
        df = pd.read_csv(prepfile )

if run_control > 0:
  
    os.remove(prepfile)
    os.remove(prepparq)

else:

    pass

#### Upload Charts

In [ ]:
def num_format(x):
    """number format without a currency"""
    return "{:,.0f}".format(x)

if run_control > 0:

    sumdata = df.groupby(['Business Unit']).sum()
    sumdata = sumdata.reset_index()
    plt.bar(sumdata['Business Unit'], sumdata['Earned Revenues net of Taxes']/1000)
    plt.title('Earned Revenue net of Taxes by BU in thousands LC')
    plt.show()

    countdata = df.groupby(['Business Unit']).count()
    countdata = countdata.reset_index()
    plt.bar(countdata['Business Unit'], countdata['Earned Revenues net of Taxes'])
    plt.title('Row Counts by BU')
    plt.show()

else:
    print("Charts Skipped")

#### Raw files - extensive audit, best to skip unless troubleshooting, may crash a browser

In [ ]:
# setup for json only
# if run_control > 1:
#     for f in pathfiles:
#         with open(f, 'r') as json_file:
#             json_object = json.load(json_file)
#         print(json.dumps(json_object, indent=1))
#
# else:
#     print("Audit Report 1 Skipped")

#### Output Processes
- Define File names.
- If existing csv is found, archive it in "raw_data_file_history" folder.
- Save new csv.

In [ ]:
if run_control > 0:
    
    file = f'{str(percode)}.csv'
    pfile = f'{str(percode)}.parquet'
    histfile = f'{str(percode)}_{inst_datetime}.csv'

    try:
        os.rename(os.path.join(str(rt_path), file), os.path.join(str(rt_path), "logs/raw_data_file_history/" , histfile))
    except FileNotFoundError:
        pass

    df.to_csv(os.path.join(str(rt_path), file), ',', index=False , encoding='utf-8-sig')
    df.to_parquet(os.path.join(str(rt_path), pfile), engine = "pyarrow")

else:
    
    pass

- Log update record with commit message and time.
- Determine commit message and make new entry

In [ ]:
log = os.path.join(str(rt_path), f'logs/pipline_log.csv' )

try:
    log_df = pd.read_csv(log)
except NameError:
    print("No log file present")

# determine if running from notebook or Excel controller and set commit message.
if sys.argv[1] == "-f":
    message = commit_message
else:
    piperunner = '//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/_To' \
                 'ols/GitHub/Production Repositories/Pipeline Runner/Pipeline Runner.xlsm'
    wb = openpyxl.load_workbook(piperunner)
    sh = wb['Pipeline Runner']
    message = sh['f13'].value

# find username
username = getpass.getuser()

# record timestamp
datetime = inst_datetime
dateformat = datetime[4:8] + '-' + datetime[:2]  + '-' + datetime[2:4]  + '-' + datetime[8:]

# add new line
newline = { "run_message" : [message ],  "datetime" :[dateformat], "user" : [username ]}
newline = pd.DataFrame.from_dict(newline)

# make new file
log_df = log_df.append(newline[0:])
log_df
log_df.to_csv(log, ',', index=False , encoding='utf-8-sig')


#### Store the DataFrame for other noteboks to use

In [ ]:
read_dc_df = df

%store read_dc_df
